<img style="float:left" width="70%" src="pics/escudo_COLOR_1L_DCHA.png">
<img style="float:right" width="15%" src="pics/PythonLogo.svg">
<br style="clear:both;">

# Evaluación de la base de datos *Drug classification*

En este *Notebook* se debe realizar el trabajo para obtener la tasa de clasificación sobre la base de datos de **Kaggle** de *clasificación de drogas*, utilizando el método de clasificación de **Support Vector Machines** (**SVM**), con parámetros $(C,\gamma)$ obtenidos mediante *búsqueda en rejilla* (*grid search*).

## Autor
- Pedro Latorre Carmona

### Curso
- 2022-2023

Esta base de datos la podéis encontrar en:

https://www.kaggle.com/datasets/prathamtripathi/drug-classification

No obstante, en el fichero comprimido de la práctica, tenéis el correspondiente fichero *.csv* en la carpeta *DrugClassification*.

Cuando abráis dicho fichero, por ejemplo, con **openoffice**, veréis que tiene el siguiente aspecto:

<img src="pics/DCHowItLooksLike.png">

Lo primero que tenéis que hacer es convertir aquellas columnas que tengan valores no numéricos, a equivalentes numéricos con los que poder trabajar. Eso incluye la última columna, que debe ser considerada como la columna de las etiquetas. 

El conjunto de los objetivos es el mismo que en el caso de la práctica 2 de primera convocatoria, y se evaluará de la misma forma.

**IMPORTANTE**

Tenéis que hacer el mismo tipo de estudio que en el caso de la práctica 2 de la primera convocatoria, pero en este caso, para esta base de datos.

In [190]:
'''
Importación de librerías
'''
import pandas as pd
import numpy as np
import pickle
import os

from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline

In [191]:
'''
Datos
'''

path = "./DrugClassification"
filename = path + os.sep + "DrugClassification200patients.csv"

### Carga del fichero csv como un *data frame*

In [192]:
# Utilizad la opción "pd.read_csv"
df = pd.read_csv(filename)

In [193]:
# Visualizad el "data frame" usando la opción "display"
display(df)

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


## Creación de los conjuntos **X** e **y**

Una vez creado el data frame, tenéis que crear los conjuntos $\mathbf{X}$ e $\mathbf{Y}$, del que luego se crea sus correspondientes conjuntos de **entrenamiento** y **test**. 

In [194]:
# Transformación de los valores no numéricos a numéricos
encoder = LabelEncoder()
# Transformación de la columna "Sex"
encoder.fit(df["Sex"])
df["Sex"] = encoder.transform(df["Sex"])
# Transformación de la columna "BP"
encoder.fit(df["BP"])
df["BP"] = encoder.transform(df["BP"])
# Transformación de la columna "Cholesterol"
encoder.fit(df["Cholesterol"])
df["Cholesterol"] = encoder.transform(df["Cholesterol"])
# Transformación de la columna "Drug"
encoder.fit(df["Drug"])
df["Drug"] = encoder.transform(df["Drug"])

y = df["Drug"].values
caract_cols = ["Age", "Sex", "BP", "Cholesterol", "Na_to_K", "Drug"]
X_all = df[caract_cols].values

In [195]:
print(X_all.shape)

(200, 6)


In [196]:
'''
Listado de datos y nombres
'''

datasets = [(X_all,y)]
dataset_names = ["Data All"]

In [197]:
'''
Definición del espacio de búsqueda para la optimización de los parámetros de SVM
'''

# Para definir el rango de "C" y de "gamma", tenéis que usar la opción "np.logspace", cubriendo, para "C", 
# desde 1.0e-2 hasta 1.0e+10, y para "gamma", desde 1.0e-9, hasta 1.0e+3.

C_range = np.logspace(-2, 10, num=13)
gamma_range = np.logspace(-9, 3, num=13)

param_grid_svm = dict(gamma=gamma_range, C=C_range)
nested_cv = 5

grid_svm = GridSearchCV(SVC(), param_grid=param_grid_svm, cv=nested_cv)

In [198]:
# Aquí se muestra el rango de valores a considerar
C_range, gamma_range

(array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
        1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10]),
 array([1.e-09, 1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02,
        1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]))

In [199]:
'''
Definición del espacio de búsqueda para MLP
'''
alpha_range = np.logspace(-5, -1, 5)
hidden_layer_sizes_range=[(50,),(100,),(200,),(500,),(1000,)]

param_grid_mlp = dict(alpha=alpha_range, hidden_layer_sizes=hidden_layer_sizes_range)


grid_mlp = GridSearchCV(MLPClassifier(max_iter=1000,
                                      early_stopping=True), param_grid=param_grid_mlp, cv=nested_cv)

In [200]:
'''
Conjunto de clasificadores usados, así como sus nombres.
'''

cls_names = ["SVM","MLP"]

classifiers = [
    make_pipeline(StandardScaler(), grid_svm),
    make_pipeline(StandardScaler(), grid_mlp)]

In [201]:
# Método que ejecuta los clasificadores y devuelve las etiquetas predichas correspondientes.

from sklearn.model_selection import train_test_split

def predictions(model,X_train,y_train,X_test,y_test):    
    
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    return y_test, y_pred

In [202]:
def predictions_model(X_train,y_train,X_test, y_test,model):
        '''
        Predicciones con un modelo y un conjunto de datos (X e y), para obtener posteriormente las medidas que se quieren
        
        Parámetros
        ----------
        X: numpy.array
            Conjunto (características)
        Y: numpy.array
            Dataset (etiquetas)
        model: scikit_model
            modelo a entrenar
        num_folds: int
            Número de "particiones" de la validación cruzada ("k-fold" cross validation)
        
        Devuelve
        -------
        array 
            array de predicciones
        '''
        print('\t'+str(model)[:20], end=' - ')
        y_test,preds = predictions(model,X_train,y_train,X_test,y_test)
        print('OK')
        
        return y_test,preds

In [203]:
def run_all_save(filename):
    '''
    Realiza la validación cruzada de todos los modelos y conjuntos de datos.
        
        
    Parámetros
    ----------
    num_folds: int
        Igual que antes
    filename: string
        Nombre del fichero que guardará las "predicciones"
        
        
    El par X_train, y_train son los atributos y clases del conjunto de entrenamiento (70% de los ejemplos)
    El par X_test, y_test son los atributos y clases del conjunto de test (30% de los ejemplos)

    stratify (estratificar) significa que se quiere que haya la misma proporcion de cada una de las clases
    tanto en entrenamiento como en test, es decir, no es una partición completamente aleatoria.
    
    ''' 
    
    all_preds = {}

    for dataset,dataset_name in zip(datasets, dataset_names):
        print(dataset_name)
        X,y = dataset
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        
        for model,cls_name in zip(classifiers,cls_names):
            print(cls_name)
            y_test,preds = predictions_model(X_train,y_train,X_test,y_test,model)
            all_preds[(dataset_name,cls_name)]=(y_test,preds)

    all_preds["cls_names"]=cls_names
    all_preds["dataset_names"]=dataset_names

    with open(filename, 'wb') as fp:
         pickle.dump(all_preds, fp)   

In [204]:
'''
All the predictions are going to be saved in a Python dictionary for 
further analysis.
'''

filename = 'PrediccionesDrugClassification.obj'

In [205]:
# Run the experiments

run_all_save(filename)

Data All
SVM
	Pipeline(steps=[('st - OK
MLP
	Pipeline(steps=[('st - OK


# Análisis de los resultados

Si los experimentos se han realizado previamente, sólo es necesario ejecutar el *notebook* desde esta parte. 

Los resultados se *cargarían* desde el disco duro.

In [206]:
import pickle
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

In [207]:
# Función que debe evaluar los resultados de clasificación.

def evalua(y_test, y_pred):
  return accuracy_score(y_test, y_pred)

In [ ]:
evalua(y_test, y_pred)

In [182]:
def conf_mat_df(cm,labels):
    '''
    Creación de una matriz de confusión en un DataFrame
        
        
    Parámetros
    ----------
    cm: ndarray 2D
        matriz de confusión
    labels: lista
        Lista de nombres de clase
        
    Return DataFrame
    -------
    
    ''' 

    return (pd.DataFrame(cm,index=labels, columns=labels)
          .rename_axis("actual")
          .rename_axis("predicted", axis=1))

In [183]:
def get_results(filename):
    '''
    Carga el fichero con las predicciones.
    Calcula la "accuracy", la matriz de confusión, y otras. 
        
        
    Parámetros
    ----------
    filename: string
        Nombre del fichero que guarda las predicciones
        
    Return
    diccionario
        Un diccionario de pares key:values
    -------
    
    ''' 

    with open(filename, 'rb') as fp:
        all_preds = pickle.load(fp)

    cls_names = all_preds.pop("cls_names")
    dataset_names = all_preds.pop("dataset_names")

    data_cls_pairs = list(all_preds.keys())
    data_cls_pairs.sort()

    results = {}


    acc_df = pd.DataFrame(index=dataset_names, columns=cls_names)

    ## A DataFrame is created to store the accuracy in each clase
    for dataset in dataset_names:
        results[(dataset,"acc")] = pd.DataFrame(columns=cls_names)


    for dataset_name,cls_name in data_cls_pairs:

        #print(dataset_name,cls_name)
        y_true, y_pred = all_preds[(dataset_name,cls_name)]
        labels = list(np.unique(y_true))

        acc = evalua(y_true, y_pred)
        # Fill accuracy dataframe
        acc_df.at[dataset_name,cls_name]=acc

        # Get conf_mat
        cm = confusion_matrix(y_true, y_pred)
        cm_df = conf_mat_df(cm,labels)
        results[(dataset_name,cls_name,"cm")] = cm_df
        
        # Get classification report
        report = classification_report(y_true, y_pred, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        results[(dataset_name,cls_name,"report")] = report_df

        # Acc per class
        cm_dig = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm_dig = cm_dig.diagonal()

        dfi = results[(dataset_name,"acc")]
        dfi[cls_name]=pd.Series(cm_dig,labels)    
        results[(dataset_name,"acc")]=dfi.copy()


    results["Acc"] = acc_df
    return results
        
        
results = get_results(filename)

In [184]:
df_total = results["Acc"].astype(float)
df_conf = results[("Data All","SVM","cm")].astype(float)
df_report = results[("Data All","SVM","report")].astype(float)

In [185]:
df_total

,SVM,MLP
Data All,1.0,0.65


In [186]:
df_conf

predicted,0,1,2,3,4
actual,,,,,
0,28.0,0.0,0.0,0.0,0.0
1,0.0,9.0,0.0,0.0,0.0
2,0.0,0.0,3.0,0.0,0.0
3,0.0,0.0,0.0,5.0,0.0
4,0.0,0.0,0.0,0.0,15.0


In [187]:
df_report.round(4)[["precision","recall","f1-score"]]

,precision,recall,f1-score
0,1.0,1.0,1.0
1,1.0,1.0,1.0
2,1.0,1.0,1.0
3,1.0,1.0,1.0
4,1.0,1.0,1.0
accuracy,1.0,1.0,1.0
macro avg,1.0,1.0,1.0
weighted avg,1.0,1.0,1.0
